In [1]:
! touch submission.csv

In [2]:
# environment setup

import os
import sys

os.environ["__KAGGLE__"] = "1"
sys.path.append("/kaggle/input/bc21-code/")

In [10]:
import numpy as np
import pandas as pd
import tensorflow_addons as tfa
from sklearn.metrics import f1_score
from tensorflow import keras
from tqdm import tqdm

from lib.utils import fix_random_seed, read_json
from src.config import c
from src.data_utils import (
    boost_multiple_occurences,
    geofilter_predictions,
    normalize_soundscapes_df,
    predictions_to_text_labels,
    read_soundscapes_info,
)
from src.generator import Generator
from src.geo_filter import filters as geo_filters
from src.models import Div, SinCos, YMToDate
from src.services import get_msg_provider, get_wave_provider

In [4]:
IN_KAGGLE_SUBMIT = os.environ["KAGGLE_KERNEL_RUN_TYPE"] != "Interactive"

IN_CSV = (
    "/kaggle/input/birdclef-2021/test.csv"
    if IN_KAGGLE_SUBMIT
    else "/kaggle/input/birdclef-2021/train_soundscape_labels.csv"
)

MODEL = "/kaggle/input/bc21-models/B1_nrsw_2.h5"
GEOFILTER = "all-500mi-last_5y-1mo_tolerance"

In [5]:
# metadata from model training run
meta = read_json(MODEL.replace(".h5", ".json"))

# copy kaggle-specific configs
for k in [
    "WORK_DIR",
    "CACHE_DIR",
    "COMPETITION_DATA",
    "SRC_DATA_DIRS",
    "CACHE_AUDIO_FRAGMENTS",
]:
    meta["config"][k] = c[k]

In [6]:
# prepare soundscapes df
df = pd.read_csv(IN_CSV)
df = normalize_soundscapes_df(df, quiet=True, seconds=5)

In [7]:
# load model
model = keras.models.load_model(
    MODEL,
    custom_objects={
        "SinCos": SinCos,
        "Div": Div,
        "YMToDate": YMToDate,
    },
)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA Tesla V100-DGXS-32GB, compute capability 7.0


In [8]:
# create generator

input_shape = model.get_layer("i_msg").input_shape[0][1:]
wave_p = get_wave_provider(meta["config"])
msg_p = get_msg_provider(
    meta["config"], n_mels=input_shape[0], time_steps=input_shape[1]
)

generator = Generator(
    df=df,
    shuffle=False,
    augmentation=None,
    rating_as_sw=False,
    rareness_as_sw=False,
    msg_provider=msg_p,
    wave_provider=wave_p,
    msg_as_rgb=(3 == input_shape[-1]),
    geo_coordinates_bins=meta["config"]["GEO_COORDINATES_BINS"],
    batch_size=1,
)

/usr/local/lib/python3.8/dist-packages/torchaudio/functional/functional.py:357: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (380) may be set too high. Or, the value for `n_freqs` (1025) may be set too low.
  warnings.warn(


In [9]:
# predict
Y_pred = model.predict(
    x=generator,
    use_multiprocessing=False,
    verbose=0 if IN_KAGGLE_SUBMIT else 1,
)

2400/2400 [==============================] - 121s 46ms/step


In [12]:
df["_y_pred"] = list(map(lambda x: x, Y_pred))

In [25]:
# boost

BOOST_COEF = 2.4
MAX_BOOST_COEF = 2.4 * 5
THRESHOLD = 0.5


df = boost_multiple_occurences(
    df=df,
    labels=meta["labels"],
    pred_col="_y_pred",
    out_col="_y_pred_boosted",
    boost_coef=BOOST_COEF,
    max_boost_coef=MAX_BOOST_COEF,
    threshold=THRESHOLD,
)

# geofiltering

if GEOFILTER is not None:

    yp = geofilter_predictions(
        df=df,
        Y_pred=np.array(list(df["_y_pred"])),
        site_labels=geo_filters[GEOFILTER],
        labels=meta["labels"],
        downgrade_const=0.0,
    )
    df["_y_pred_geof"] = list(map(lambda x: x, yp))

    yp = geofilter_predictions(
        df=df,
        Y_pred=np.array(list(df["_y_pred_boosted"])),
        site_labels=geo_filters[GEOFILTER],
        labels=meta["labels"],
        downgrade_const=0.0,
    )
    df["_y_pred_geof_boosted"] = list(map(lambda x: x, yp))

In [27]:
# convert predictions to words

labels_cols = []

for col in [
    "_y_pred",
    "_y_pred_boosted",
    "_y_pred_geof",
    "_y_pred_geof_boosted",
]:
    if col in df:

        labels_pred = predictions_to_text_labels(
            np.array(list(df[col])),
            meta["labels"],
            default_label="nocall",
            max_labels=None,
            priority_to_nocall=False,
            threshold=THRESHOLD,
        )

        labels_col = col.replace("_y_pred", "birds")
        labels_cols.append(labels_col)
        df[labels_col] = labels_pred

        if not IN_KAGGLE_SUBMIT:

            df[df["_primary_labels"] != df["birds"]][
                ["filename", "_primary_labels"] + labels_cols
            ].to_csv("wrong.csv", index=False)

            print(
                labels_col,
                "f1",
                f1_score(
                    df["_primary_labels"],
                    df[labels_col],
                    labels=meta["labels"],
                    average="micro",
                ),
            )

birds f1 0.9393600358021928
birds_boosted f1 0.9580838323353293
birds_geof f1 0.9393600358021928
birds_geof_boosted f1 0.9580838323353293


In [28]:
# df[["row_id", "birds"]].to_csv("submission.csv", index=False)